## `asyncio` syntax

A coroutine is a special type of function that can **pause and resume its execution**. Unlike a regular function that runs from start to finish without interruption, a coroutine can temporarily stop itself when it encounters an operation that takes time to complete. This includes reading a file from a disk or making a network request. This “pause” allows the program to do other things instead of waiting idly.

Imagine you're cooking dinner and the recipe calls for boiling water. Instead of just staring at the pot until it boils (a regular function), you put the lid on and walk away to chop some vegetables (the program doing other work). When the water finally boils, you can come back and continue with the next step of the recipe—like adding pasta. The coroutine is like the cook who can efficiently multitask by pausing one task to start another.

When the long-running operation is done, the coroutine is “woken up” and resumes exactly where it left off, completing the rest of its code. This ability to pause and resume is the key to achieving **concurrency**, allowing a single program to manage many tasks efficiently without blocking or freezing.

### Creating coroutines using `async` syntac


Creating a coroutine is simple, just like defining a regular function, but you use the keyword **`async`** before **`def`**. This small change is what tells Python the function is a coroutine, allowing it to be paused and resumed.


```python
async def my_coroutine():
    print("Hello coroutine")
```

To run a coroutine in Python, you can't simply call it like a regular function. Coroutines need to be executed by an event loop, which is a special program that manages and schedules the execution of asynchronous tasks. The simplest and most common way to do this is by using the `asyncio.run()` function.

In [10]:
import asyncio
import time

# A coroutine is defined with 'async def'.
async def my_coroutine(name):
    """
    This is a coroutine that simulates an I/O-bound task.
    """
    print(f"[{time.time():.2f}] Coroutine '{name}' started.")
    
    # The 'await' keyword pauses this coroutine and gives control back to the event loop.
    # The event loop can then run other tasks while 'my_coroutine' is waiting.
    await asyncio.sleep(2)  # Simulates waiting for 2 seconds (e.g., a network request)
    
    print(f"[{time.time():.2f}] Coroutine '{name}' resumed and finished.")

# The 'main' function is a coroutine that serves as the entry point for the program.
# It is also defined with 'async def'.
async def main():
    """
    The main entry point for the asynchronous program.
    """
    print("Program starting...")
    
    # We call our coroutine here, but this does not run it immediately.
    # It simply returns an "awaitable" object (a coroutine object).
    coro = my_coroutine("First Task")
    
    # The 'await' keyword is used to execute the coroutine.
    # The program will pause here until 'coro' has finished running.
    await coro
    
    print("Program finished.")

**You can not run this coroutine like this**

In [11]:
main()

<coroutine object main at 0x7f7540526980>

**Instead, you should create an event loop like this**

```python
import asyncio

result = asyncio.run(main())
```

In jupyter notebooks an event loop is created by default and we can use `await` directly

In [12]:
await main()

Program starting...
[1757760448.00] Coroutine 'First Task' started.
[1757760450.00] Coroutine 'First Task' resumed and finished.
Program finished.


## Understanding `asyncio.run()`

The most important thing about **`asyncio.run()`** is that it serves as the primary **entry point** for an `asyncio` application. Think of it as the **`main()`** function in a traditional program. It's the one call that kicks off everything else. It's not designed to run multiple, unrelated coroutines at the same time. Instead, it runs a single, top-level coroutine, and that one coroutine is responsible for launching and managing all other concurrent tasks.

---

### The Role of the Entry Coroutine

The single coroutine you pass to `asyncio.run()` is your application's starting point.  This main coroutine will then use `asyncio` functions like **`asyncio.gather()`** or **`asyncio.create_task()`** to create and execute other coroutines. This is how you harness `asyncio`'s **concurrency**—by delegating tasks from the main coroutine to other, more specific ones. This design ensures that all concurrent operations are properly managed within a single, cohesive event loop created by `asyncio.run()`, preventing common issues with unmanaged asynchronous code. As you build more complex `asyncio` applications, you'll consistently use this pattern, with `asyncio.run()` as the reliable starting point for all your asynchronous logic.

### The `await` keyword

The `await` keyword pauses the execution of a coroutine until a specific asynchronous operation is complete, allowing the program to do other work in the meantime. It can only be used inside a coroutine function (defined with `async def`) and must be followed by an awaitable object, such as another coroutine, a task, or a future.

The `await` keyword is the modern, more readable successor to the `yield from` keyword in asynchronous programming. Before Python 3.5 introduced `async` and `await`, `yield from` was the primary way to delegate control to a nested coroutine. Although `yield from` could be used for other purposes, in the context of coroutines, it allowed one generator (which acted as a coroutine) to hand over control to another.

| | **`await`** | **`yield from`** |
| :--- | :--- | :--- |
| **Purpose** | Used specifically for asynchronous operations. | A general-purpose tool for delegating to sub-generators. |
| **Readability** | Explicitly conveys the intent to "wait for" an async result. | Less clear about its asynchronous purpose. |
| **Syntax** | `await coro()` | `yield from coro()` |
| **Used in** | Coroutines defined with `async def`. | Generators and pre-Python 3.5 coroutines. |

Essentially, the introduction of `await` made the asynchronous code more semantic and easier to understand, separating the specific act of waiting for an async result from the more general `yield from` delegation mechanism.


`await` pauses a coroutine's execution by yielding control to the `asyncio` event loop. The event loop then uses this opportunity to run other tasks, effectively multitasking. When the awaited operation is complete, the event loop resumes the paused coroutine from where it left off.

-----

### Example: Awaiting an `asyncio.sleep()`

Let's look at a simple example using `asyncio.sleep()`, which is an awaitable object that simulates a time-consuming I/O operation (like waiting for a network response).

```python
import asyncio

async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():
    print("starting main()")
    
    # Task 1
    task1 = asyncio.create_task(
        say_after(3, 'hello')
    )
    # Task 2
    task2 = asyncio.create_task(
        say_after(1, 'world')
    )

    print("tasks created, now awaiting")
    await task1
    await task2
    
    print("finished main()")

asyncio.run(main())
```

-----

### Step-by-Step Execution

1.  **`asyncio.run(main())`** starts the event loop and begins executing the `main()` coroutine.

2.  `main()` prints "starting main()" and then creates two tasks: `task1` (to run `say_after(3, 'hello')`) and `task2` (to run `say_after(1, 'world')`). These tasks are now scheduled to run.

3.  `main()` prints "tasks created, now awaiting".

4.  The code reaches **`await task1`**. This is the key moment. The `main()` coroutine **pauses** its execution and yields control back to the **event loop**.

5.  **The Event Loop takes over.** Since `main()` is paused, the event loop looks for other scheduled tasks to run. It finds `task1` and `task2` waiting. It starts running both of them concurrently.

6.  Inside `task2`, the code hits **`await asyncio.sleep(1)`**. `task2` pauses and yields control back to the event loop.

7.  The event loop continues to monitor both tasks. After **1 second** passes, the event loop notices that `task2`'s sleep is complete. It **resumes** `task2` from where it left off. `task2` prints "world" and finishes.

8.  The event loop continues to monitor `task1`. After **3 seconds** pass (from the beginning), it notices that `task1`'s sleep is complete. It **resumes** `task1`, which then prints "hello" and finishes.

9.  Now that both `task1` and `task2` are finished, the event loop returns control to the `main()` coroutine, which had been paused at `await task1`. Since `task1` is now complete, `main()` continues.

10. The code then reaches **`await task2`**. Since `task2` is already complete, this `await` returns immediately without pausing.

11. `main()` prints "finished main()" and the program ends.

The final output is:

```
starting main()
tasks created, now awaiting
world
hello
finished main()
```

This sequence demonstrates that the `await` keyword doesn't block the entire program. Instead, it temporarily suspends a single coroutine to let other tasks run, making efficient use of the CPU.

## Creating `Tasks`

A `Task` in `asyncio` is a fundamental building block for running coroutines concurrently on an event loop. A `Task` is a **wrapper around a coroutine** that schedules its execution. Creating a `Task` doesn't run the coroutine immediately; it just puts it on the event loop's to-do list, allowing the event loop to manage its execution alongside other tasks.

-----

### What a `Task` Does

The primary purpose of a `Task` is to enable true concurrency. When you have multiple coroutines you want to run at the same time, you wrap each one in a `Task`. This allows the event loop to switch between them whenever an `await` is encountered.

Think of an `asyncio.Task` as a lightweight thread or a job ticket.  You don't manage the execution yourself; you hand the "job" (the coroutine) to the event loop by creating a task. The event loop's job is to make sure all these tickets are processed efficiently.

A key benefit of `Task` is that it allows for **cancellation**. You can programmatically cancel a `Task`, and `asyncio` will handle the cleanup. This is crucial for managing long-running operations.

-----

### Creating and Running Tasks

You typically create a task using `asyncio.create_task()`. The function takes a coroutine object and returns a `Task` object.

Here's an example:

```python
import asyncio

async def worker(name, delay):
    print(f"Task {name}: starting...")
    await asyncio.sleep(delay)
    print(f"Task {name}: finished.")
    return f"Result of {name}"

async def main():
    print("Main: creating tasks...")

    # Create tasks without immediately awaiting them
    task1 = asyncio.create_task(worker('A', 3))
    task2 = asyncio.create_task(worker('B', 1))

    print("Main: tasks are scheduled, now waiting for them to complete...")

    # Await the results of the tasks concurrently
    result1 = await task1
    result2 = await task2

    print(f"Main: Got results: {result1}, {result2}")

# Entry point of the application
asyncio.run(main())
```

#### Step-by-Step Breakdown:

1.  `asyncio.run(main())` starts the event loop and runs the `main()` coroutine.
2.  Inside `main()`, `asyncio.create_task()` is called for `worker('A', 3)` and `worker('B', 1)`. This **schedules** both coroutines to be run by the event loop but **does not block** `main()`.
3.  The `main()` coroutine then reaches `await task1`. This pauses `main()` and yields control back to the event loop.
4.  The event loop now has two tasks it can run: `task1` and `task2`. It switches between them as they encounter `await` calls (in this case, `await asyncio.sleep()`).
5.  `task2` finishes first (after 1 second).
6.  `task1` finishes after 3 seconds.
7.  Once `task1` and `task2` are both done, the event loop resumes `main()`. `main()` gets the results from the tasks and prints them.

Using `asyncio.create_task()` is the preferred way to run a coroutine in the background, as it gives you a `Task` object that you can use to manage the coroutine's lifecycle, get its result, or cancel it.

## Canceling a `Task`

To cancel an `asyncio` task, you call the **`.cancel()`** method on the `Task` object. This doesn't stop the task immediately but instead schedules it for cancellation by raising an **`asyncio.CancelledError`** inside the coroutine at its next `await` point.

-----

### How It Works and What to Expect

1.  **Call `.cancel()`**: When you call `task.cancel()`, you're essentially sending a cancellation signal. The task's `_asyncio.CancelledError` flag is set, and the event loop is notified.

2.  **`CancelledError` Injection**: The event loop continues running until the cancelled task next `awaits` something. At that point, the `asyncio.CancelledError` exception is injected and raised within the coroutine.

3.  **Coroutine Handles Exception**: The coroutine should be prepared to catch this exception. A `try...except asyncio.CancelledError` block allows the coroutine to run any necessary cleanup code, such as closing files or database connections, before it terminates. If a coroutine doesn't handle the exception, it will propagate up and cause the task to terminate abruptly.

4.  **Awaiting the Canceled Task**: To ensure the cancellation process is complete, you can `await` the task you just cancelled. This `await` will re-raise the `CancelledError`, which you can catch to confirm the task is done.

-----

### Example

Here's a simple example showing how to cancel a task and handle the cancellation within the coroutine.

```python
import asyncio
import time

async def worker_task(name):
    print(f"[{name}] Task started.")
    try:
        # Simulate a long-running, awaitable operation
        await asyncio.sleep(10)
        print(f"[{name}] Task finished successfully.")
    except asyncio.CancelledError:
        print(f"[{name}] Cancellation received. Performing cleanup...")
        # Optional: A brief sleep to simulate cleanup time
        await asyncio.sleep(1)
        print(f"[{name}] Cleanup complete. The task is now cancelled.")
    finally:
        print(f"[{name}] Exiting gracefully.")

async def main():
    # Create the task
    print("[Main] Creating and scheduling the task.")
    my_task = asyncio.create_task(worker_task("Worker-1"))

    # Wait for a brief moment before cancelling
    print("[Main] Waiting 2 seconds before cancelling.")
    await asyncio.sleep(2)

    # Cancel the task
    print("[Main] Requesting task cancellation.")
    my_task.cancel()

    # Wait for the task to fully terminate
    try:
        print("[Main] Awaiting task to confirm cancellation.")
        await my_task
    except asyncio.CancelledError:
        print("[Main] Task has been confirmed as cancelled.")

asyncio.run(main())
```

#### Output:

```
[Main] Creating and scheduling the task.
[Worker-1] Task started.
[Main] Waiting 2 seconds before cancelling.
[Main] Requesting task cancellation.
[Worker-1] Cancellation received. Performing cleanup...
[Worker-1] Cleanup complete. The task is now cancelled.
[Worker-1] Exiting gracefully.
[Main] Awaiting task to confirm cancellation.
[Main] Task has been confirmed as cancelled.
```

As you can see, the `await asyncio.sleep(10)` in `worker_task` is interrupted, the `except asyncio.CancelledError` block runs, and the program exits cleanly, demonstrating that cancellation is a cooperative process.

### You can not cancel a `Task` when it is running plain python

Something important to note about cancellation is that a `CancelledError` can only be thrown from an `await` statement. This means that if you call `cancel()` on a task while it is executing plain Python code, that code will run until completion. The `CancelledError` will only be raised when the task hits its next `await` statement. Calling `cancel()` won't magically stop the task in its tracks; it will only stop the task if you're currently at an `await` point or its next `await` point.

-----

### How Cancellation Works: The Await Point

Cancellation in `asyncio` is **cooperative**, not preemptive. A task must "cooperate" by reaching an `await` point to allow the `CancelledError` to be raised. This design choice is crucial for maintaining the integrity of the program's state. It ensures that a task can't be abruptly terminated while in the middle of a critical operation, like updating a database or writing a file, which could lead to data corruption.

Imagine the event loop as a single thread of execution. When a task is running plain Python code (CPU-bound work), it holds control of this thread and will not give it up until it hits an `await` statement.  This `await` is the only opportunity for the event loop to regain control and inject the cancellation signal.

### Example

Let's illustrate this with an example that includes both CPU-bound work and an `await` call.

```python
import asyncio
import time

def cpu_intensive_operatin():
    print("Task: Starting heavy computation...")
    start_time = time.time()
    # This loop runs plain Python code and does not yield to the event loop.
    while time.time() - start_time < 10:
        # Simulate CPU-bound work
        _ = 1 + 1
    print("Task: Heavy computation finished.")

async def worker_with_await():
    print("Worker: Starting...")
    cpu_intensive_operatin()  # Await a task that does CPU-bound work
    print("Worker: First part complete, now awaiting something else...")
    try:
        await asyncio.sleep(5)  # The cancellation will happen here
    except asyncio.CancelledError:
        print("Worker: Caught CancelledError. Cleanup complete.")
        raise  # Re-raise to signal cancellation is complete
    finally:
        print("Worker: Exiting gracefully.")

async def main():
    task = asyncio.create_task(worker_with_await())
    print("Main: Task created. Waiting 1 second before cancelling...")
    await asyncio.sleep(1)
    
    print("Main: Calling cancel() on the task.")
    task.cancel()
    
    try:
        await task
    except asyncio.CancelledError:
        print("Main: Task was successfully cancelled.")

await main()
```

#### Breakdown of Execution:

1.  `main()` creates `worker_with_await()` as a task.
2.  `main()` waits for 1 second (`await asyncio.sleep(1)`).
3.  `main()` calls `task.cancel()`.
4.  The `CancelledError` is now pending, but it's not raised yet because `worker_with_await` is running its `cpu_intensive_operation`, which does not have any `await` statements. The CPU-intensive loop runs for its full 3 seconds.
5.  After the `cpu_intensive_task` finishes, the `worker_with_await` coroutine hits its next `await` point: `await asyncio.sleep(5)`.
6.  It is at this **precise moment** that the pending `CancelledError` is finally raised. The `try...except` block catches it, prints the message, and the task ends gracefully. The `asyncio.sleep(5)` never actually runs.
7.  Finally, `await task` in `main()` completes, and the program exits, confirming the cancellation.

**⚠️ Note:** This example clearly shows that `cancel()` is a **request**, not a command. The task itself decides when to honor that request by hitting an `await` point.

In [7]:
import asyncio
import time

def cpu_intensive_operatin():
    print("Task: Starting heavy computation...")
    start_time = time.time()
    # This loop runs plain Python code and does not yield to the event loop.
    while time.time() - start_time < 10:
        # Simulate CPU-bound work
        _ = 1 + 1
    print("Task: Heavy computation finished.")

async def worker_with_await():
    print("Worker: Starting...")
    cpu_intensive_operatin()  # Await a task that does CPU-bound work
    print("Worker: First part complete, now awaiting something else...")
    try:
        await asyncio.sleep(5)  # The cancellation will happen here
    except asyncio.CancelledError:
        print("Worker: Caught CancelledError. Cleanup complete.")
        raise  # Re-raise to signal cancellation is complete
    finally:
        print("Worker: Exiting gracefully.")

async def main():
    task = asyncio.create_task(worker_with_await())
    print("Main: Task created. Waiting 1 second before cancelling...")
    await asyncio.sleep(1)
    
    print("Main: Calling cancel() on the task.")
    task.cancel()
    
    try:
        await task
    except asyncio.CancelledError:
        print("Main: Task was successfully cancelled.")

await main()

Main: Task created. Waiting 1 second before cancelling...
Worker: Starting...
Task: Starting heavy computation...
Task: Heavy computation finished.
Worker: First part complete, now awaiting something else...
Main: Calling cancel() on the task.
Worker: Caught CancelledError. Cleanup complete.
Worker: Exiting gracefully.
Main: Task was successfully cancelled.


## Setting a `timeout` for a `Task`

You use `asyncio.wait_for` to set a time limit on an asynchronous operation. It's a high-level `asyncio` function that runs a coroutine and cancels it if it doesn't finish within the specified timeout. This is essential for preventing a single operation from blocking your entire application indefinitely.

-----

### How to Use `asyncio.wait_for`

The basic syntax is `await asyncio.wait_for(coro, timeout)`, where:

  - `coro` is the **awaitable object** you want to run with a time limit (a coroutine, a `Task`, or a `Future`).
  - `timeout` is a **number** of seconds (an `int` or `float`) after which the operation will be cancelled. If the `timeout` is `None`, the function will wait forever.

If the operation completes within the timeout, `asyncio.wait_for` returns its result. If the timeout expires, it cancels the task and raises an **`asyncio.TimeoutError`**. This means you must use a `try...except` block to handle a potential timeout.

### Example 1: Operation Completes in Time

In this example, a "fast" task finishes before the timeout, so `wait_for` returns the result without raising an error.

```python
import asyncio

async def fast_operation():
    print("Fast operation is starting...")
    await asyncio.sleep(1) # This is less than the 3-second timeout
    return "Operation complete!"

async def main():
    try:
        print("Waiting for fast operation...")
        result = await asyncio.wait_for(fast_operation(), timeout=3)
        print(f"Result: {result}")
    except asyncio.TimeoutError:
        print("Error: The operation timed out.")

asyncio.run(main())
```

**Output:**

```
Waiting for fast operation...
Fast operation is starting...
Result: Operation complete!
```

-----

### Example 2: Operation Exceeds the Timeout

Here, a "slow" task takes longer than the timeout, causing `wait_for` to raise a `TimeoutError`. The `try...except` block catches this error and handles the cancellation.

```python
import asyncio

async def slow_operation():
    print("Slow operation is starting...")
    try:
        await asyncio.sleep(5) # This is more than the 3-second timeout
        print("This message will not be printed because the task was cancelled.")
    except asyncio.CancelledError:
        print("Slow operation was cancelled.")
        raise # It's good practice to re-raise the exception after cleanup

async def main():
    try:
        print("Waiting for slow operation...")
        await asyncio.wait_for(slow_operation(), timeout=3)
        print("This message will not be printed.")
    except asyncio.TimeoutError:
        print("Successfully caught a TimeoutError.")

asyncio.run(main())
```

**Output:**

```
Waiting for slow operation...
Slow operation is starting...
Successfully caught a TimeoutError.
Slow operation was cancelled.
```

Notice that `asyncio.wait_for` automatically cancels the `slow_operation` when the timeout is reached. Inside the `slow_operation` coroutine, the `asyncio.CancelledError` is raised, allowing you to perform cleanup before the task terminates.

## `Futures` in `ayncio`

A `Future` in `asyncio` is a special low-level object that represents the eventual result of an asynchronous operation. Unlike a coroutine, a `Future` doesn't contain any code to be executed. It's a placeholder for a result that will be set at a later time.

Think of a **`Future` as a promise**: a promise that a result will be available in the future. You can check if the promise has been fulfilled, wait for it to be, or attach a callback function to run once it is.

-----

### Key Characteristics of a `Future`

  * **State:** A `Future` can be in one of several states: `PENDING`, `RUNNING`, `DONE`, or `CANCELLED`.
  * **Result or Exception:** Once a `Future` is marked as `DONE`, it holds either a return value (`future.result()`) or an exception (`future.exception()`).
  * **Awaits:** A `Future` is an **awaitable object**. This means you can use the `await` keyword on a `Future` to pause your coroutine until the `Future`'s result is available.

While you won't typically create `Future` objects manually in application code, they are a fundamental part of the `asyncio` framework. High-level functions like `asyncio.create_task()` and `asyncio.wait()` use `Future` objects under the hood to manage and report on the status of coroutines.

-----

### Example: Manual Creation and Use

This example shows how a `Future` can be used to coordinate between two parts of an asynchronous program: a `worker` that performs a task and a `main` coroutine that waits for the result.

```python
import asyncio

async def worker(future):
    """
    Simulates an operation that takes time and then sets a result on a Future.
    """
    print("Worker: Starting long operation...")
    await asyncio.sleep(2)  # Simulate I/O work
    print("Worker: Operation complete. Setting result on Future.")
    # Set the result on the future, fulfilling the promise
    future.set_result("Data successfully processed!")

async def main():
    # Create an empty Future object
    my_future = asyncio.Future()

    # Create a task to run the worker and pass the Future to it
    awaitable_task = asyncio.create_task(
        worker(my_future)
    )

    print("Main: Waiting for the Future to have a result...")
    # Await the Future directly. This pauses main() until the result is set.
    result = await my_future
    
    print(f"Main: Future's result is: '{result}'")

# Run the main coroutine
asyncio.run(main())
```

#### Step-by-Step Breakdown:

1.  `main()` creates an `asyncio.Future()` object called `my_future`. It's currently in a `PENDING` state.
2.  `main()` creates a task to run the `worker` coroutine and passes the `my_future` object to it.
3.  `main()` then hits `await my_future`. Since the `Future` is not yet done, `main()` **pauses**, yielding control back to the event loop.
4.  The event loop now runs the `worker` task. The worker sleeps for 2 seconds, and then calls `my_future.set_result()`. This action **fulfills the promise**, and the `Future`'s state changes from `PENDING` to `DONE`.
5.  Since the `Future` is now done, the event loop resumes the `main()` coroutine, which had been paused at `await my_future`.
6.  `await my_future` returns the result that was set on it, which is then printed.

This example, while simplified, shows the core principle: a `Future` acts as a coordination point for a result that is computed asynchronously, allowing other parts of your code to wait for it in a non-blocking manner.

## `Task`, `Future` and `Awaitable` relationships

In `asyncio`, **`Task`**, **`Future`**, and **`Awaitable`** are all interconnected concepts that work together to enable asynchronous programming. The simplest way to understand their relationship is to see them as a hierarchy: an `Awaitable` is the most general term, a `Future` is a specific type of `Awaitable`, and a `Task` is a special `Future` that wraps a coroutine.

***

### Awaitable

An **awaitable** is any object that can be used with the `await` keyword. This is the broadest category. It includes:

* **Coroutines**: Functions defined with `async def`.
* **Futures**: Objects that represent a result that will be available in the future.
* **Tasks**: A special type of `Future` used to schedule a coroutine.

You can **`await`** any of these objects. For example, `await my_coroutine()`, `await my_future`, or `await my_task` are all valid.

***

### Future

A **`Future`** (`asyncio.Future`) is an awaitable object that acts as a placeholder for an eventual result. It doesn't contain any code to be executed, but its state can be monitored. You can set a result (`future.set_result()`) or an exception (`future.set_exception()`) on it, at which point its state changes to `DONE`. This fulfills the "promise" of the future and unblocks any coroutines that are `await`ing it. 

***

### Task

A **`Task`** (`asyncio.Task`) is a subclass of `Future` and is the most common object you'll work with in `asyncio`. A `Task` is created by wrapping a coroutine with `asyncio.create_task()`. It serves two primary purposes:

1.  It **schedules the coroutine** to run on the event loop.
2.  It acts as a `Future`, allowing you to `await` it to get the coroutine's final result or exception.



Think of the relationship like this:

* **You write a coroutine** (`async def`). It's the code you want to run.
* You use **`asyncio.create_task()` to create a `Task`** that wraps your coroutine. This is how you tell the event loop to run your code concurrently.
* The `Task` itself is a **`Future`**. You can **`await`** it to get the result of the coroutine it's running.

The hierarchy is: **`Task`** is a specialized **`Future`**, and both are types of **`Awaitable`**.


<img src="./pics/futures.png" />